In [3]:
#!/usr/bin/env python
"""
Calculate SEFD for DSA-110 using CASA tools and VLA calibrator observations
"""

import numpy as np
from casatools import ms, table
from casatasks import listobs, flagdata
import matplotlib.pyplot as plt

class DSA110_SEFD_Calculator:
    def __init__(self):
        """Initialize CASA tools"""
        self.ms_tool = ms()
        self.tb = table()
        
        # VLA calibrator flux densities at common frequencies
        # These are approximate values - you should use setjy or look up precise values
        self.calibrator_fluxes = {
            'J1459+716': {'1.4GHz': 1.83, '1.5GHz': 1.77},  # Jy
            'J0542+498': {'1.4GHz': 5.41, '1.5GHz': 5.28},  # 3C147
            'J0319+415': {'1.4GHz': 13.86, '1.5GHz': 13.36}  # 3C84
        }
    
    def get_obs_info(self, msname):
        """Extract observation information from MS"""
        print(f"\nGetting observation info for {msname}")
        
        # Open the MS
        self.ms_tool.open(msname)
        
        # Get frequency information
        spw_table = self.ms_tool.getspectralwindowinfo()
        freqs = []
        bandwidths = []
        for spw in spw_table:
            freq_info = spw_table[spw]
            center_freq = freq_info['RefFreq'] + freq_info['TotalWidth']/2
            freqs.append(center_freq)
            bandwidths.append(freq_info['TotalWidth'])
        
        # Get antenna information using table tool
        self.tb.open(msname + '/ANTENNA')
        n_ant = self.tb.nrows()
        ant_names = self.tb.getcol('NAME')
        ant_positions = self.tb.getcol('POSITION')
        self.tb.close()
        
        # Get integration time
        self.ms_tool.selectinit(datadescid=0)
        self.ms_tool.select()
        data = self.ms_tool.getdata(['time'])
        int_time = np.median(np.diff(data['time']))
        
        self.ms_tool.close()
        
        return {
            'frequencies': freqs,
            'bandwidths': bandwidths,
            'n_antennas': n_ant,
            'ant_names': ant_names,
            'ant_positions': ant_positions,
            'integration_time': int_time,
            'center_freq': np.mean(freqs),
            'total_bandwidth': np.sum(bandwidths)
        }
    
    def get_calibrator_flux(self, source_name, freq_hz):
        """Get calibrator flux density at given frequency"""
        # Convert frequency to GHz
        freq_ghz = freq_hz / 1e9
        
        # For now, use simple interpolation between 1.4 and 1.5 GHz
        # In practice, you should use proper spectral models
        if source_name in self.calibrator_fluxes:
            flux_1p4 = self.calibrator_fluxes[source_name]['1.4GHz']
            flux_1p5 = self.calibrator_fluxes[source_name]['1.5GHz']
            
            # Linear interpolation (should use proper spectral index)
            flux = flux_1p4 + (flux_1p5 - flux_1p4) * (freq_ghz - 1.4) / 0.1
            return flux
        else:
            raise ValueError(f"Unknown calibrator: {source_name}")
    
    def calculate_sefd_from_calibrator(self, msname, source_name, baseline_range=None):
        """
        Calculate SEFD using calibrator observation
        
        Parameters:
        -----------
        msname : str
            Path to measurement set
        source_name : str
            Name of calibrator source
        baseline_range : tuple
            (min, max) baseline length in meters to use
        """
        print(f"\nCalculating SEFD from {source_name} observation")
        
        # Get observation info
        obs_info = self.get_obs_info(msname)
        freq_hz = obs_info['center_freq']
        
        # Get calibrator flux
        cal_flux = self.get_calibrator_flux(source_name, freq_hz)
        print(f"Calibrator flux at {freq_hz/1e9:.3f} GHz: {cal_flux:.2f} Jy")
        
        # Open MS and get visibility data
        self.ms_tool.open(msname)
        self.ms_tool.selectinit(datadescid=0)
        
        # Select data (you might want to add time/baseline selection)
        selection = {}
        if baseline_range:
            selection['uvdist'] = f'{baseline_range[0]}~{baseline_range[1]}m'
        
        self.ms_tool.select(selection)
        
        # Get visibility amplitudes
        data = self.ms_tool.getdata(['amplitude', 'flag', 'antenna1', 'antenna2', 'uvdist'])
        amps = data['amplitude']
        flags = data['flag']
        ant1 = data['antenna1']
        ant2 = data['antenna2']
        uvdist = data['uvdist']
        
        self.ms_tool.close()
        
        # Apply flags and flatten
        amps_masked = np.ma.masked_where(flags, amps)
        
        # Calculate SEFD for each baseline
        sefd_baselines = []
        
        # For each unique baseline
        unique_baselines = set(zip(ant1.flatten(), ant2.flatten()))
        
        for a1, a2 in unique_baselines:
            if a1 >= a2:  # Skip autocorrelations and duplicate baselines
                continue
                
            # Select data for this baseline
            bl_mask = (ant1 == a1) & (ant2 == a2)
            bl_amps = amps_masked[:, :, bl_mask]
            
            if bl_amps.count() == 0:
                continue
            
            # RMS of visibility amplitudes
            vis_rms = np.ma.std(bl_amps)
            
            # Mean visibility amplitude
            vis_mean = np.ma.mean(bl_amps)
            
            # SNR
            snr = vis_mean / vis_rms
            
            # SEFD for this baseline
            # SEFD = S_cal * sqrt(2 * delta_nu * tau) / SNR
            # where S_cal is calibrator flux, delta_nu is bandwidth, tau is integration time
            bandwidth = obs_info['total_bandwidth']  # Use actual bandwidth from MS
            sefd_bl = cal_flux * np.sqrt(2 * bandwidth * obs_info['integration_time']) / snr
            
            sefd_baselines.append({
                'ant1': a1,
                'ant2': a2,
                'sefd': sefd_bl,
                'snr': snr,
                'vis_mean': vis_mean,
                'vis_rms': vis_rms
            })
        
        # Calculate per-antenna SEFD
        antenna_sefd = self.calculate_per_antenna_sefd(sefd_baselines, obs_info['n_antennas'])
        
        return {
            'baseline_sefd': sefd_baselines,
            'antenna_sefd': antenna_sefd,
            'mean_sefd': np.mean([a['sefd'] for a in antenna_sefd if not np.isnan(a['sefd'])]),
            'calibrator_flux': cal_flux,
            'frequency': freq_hz
        }
    
    def calculate_per_antenna_sefd(self, baseline_sefd, n_antennas):
        """Calculate per-antenna SEFD from baseline measurements"""
        # For interferometer baselines, SEFD_baseline = sqrt(SEFD_ant1 * SEFD_ant2)
        # We need to solve for individual antenna SEFDs
        
        # Create matrix equation: log(SEFD_bl) = 0.5 * (log(SEFD_a1) + log(SEFD_a2))
        import scipy.sparse.linalg
        
        # Build sparse matrix
        n_baselines = len(baseline_sefd)
        A = np.zeros((n_baselines, n_antennas))
        b = np.zeros(n_baselines)
        
        for i, bl in enumerate(baseline_sefd):
            if not np.isnan(bl['sefd']) and bl['sefd'] > 0:
                A[i, bl['ant1']] = 0.5
                A[i, bl['ant2']] = 0.5
                b[i] = np.log(bl['sefd'])
        
        # Solve least squares
        try:
            log_sefd_ant, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
            sefd_ant = np.exp(log_sefd_ant)
        except:
            print("Warning: Could not solve for per-antenna SEFD")
            sefd_ant = np.full(n_antennas, np.nan)
        
        antenna_sefd = []
        for i in range(n_antennas):
            antenna_sefd.append({
                'antenna': i,
                'sefd': sefd_ant[i] if i < len(sefd_ant) else np.nan
            })
        
        return antenna_sefd
    
    def plot_sefd_results(self, results, output_file='sefd_results.png'):
        """Plot SEFD results"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
        # Plot per-antenna SEFD
        ant_ids = [a['antenna'] for a in results['antenna_sefd']]
        ant_sefds = [a['sefd'] for a in results['antenna_sefd']]
        
        ax1.bar(ant_ids, ant_sefds)
        ax1.axhline(results['mean_sefd'], color='r', linestyle='--', 
                    label=f'Mean SEFD: {results["mean_sefd"]:.1f} Jy')
        ax1.set_xlabel('Antenna ID')
        ax1.set_ylabel('SEFD (Jy)')
        ax1.set_title(f'Per-Antenna SEFD at {results["frequency"]/1e9:.3f} GHz')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot baseline SEFD distribution
        bl_sefds = [bl['sefd'] for bl in results['baseline_sefd'] if not np.isnan(bl['sefd'])]
        ax2.hist(bl_sefds, bins=30, alpha=0.7)
        ax2.axvline(np.median(bl_sefds), color='r', linestyle='--', 
                    label=f'Median: {np.median(bl_sefds):.1f} Jy')
        ax2.set_xlabel('Baseline SEFD (Jy)')
        ax2.set_ylabel('Number of Baselines')
        ax2.set_title('Baseline SEFD Distribution')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(output_file, dpi=150)
        print(f"Saved plot to {output_file}")
        plt.close()
    
    def analyze_holography_data(self, msname, source_name):
        """
        Analyze holography data for beam pattern and sensitivity
        This is a simplified version - full holography analysis is more complex
        """
        print(f"\nAnalyzing holography data for {source_name}")
        
        # Get observation info
        obs_info = self.get_obs_info(msname)
        
        # Open MS
        self.ms_tool.open(msname)
        self.ms_tool.selectinit(datadescid=0)
        
        # Get data including pointing info
        data = self.ms_tool.getdata(['amplitude', 'time', 'antenna1', 'antenna2'])
        
        # You would need to correlate this with antenna pointing information
        # to build the holography map
        
        self.ms_tool.close()
        
        print("Holography analysis would require antenna pointing data")
        print("This can be used to measure beam efficiency and refine SEFD calculations")
        
        return obs_info


#def main():
#    """Main execution function"""
#    # Initialize calculator
#    calc = DSA110_SEFD_Calculator()
#    
#    # Define your MS files - UPDATE THESE PATHS
#    ms_j1459 = 'J1459+716.ms'  # Replace with your actual path
#    ms_j0542 = 'J0542+498_holography.ms'  # Replace with your actual path
#    ms_j0319 = 'J0319+415_holography.ms'  # Replace with your actual path
#    
#    try:
#        # Calculate SEFD from J1459+716 (1-hour track)
#        results_j1459 = calc.calculate_sefd_from_calibrator(ms_j1459, 'J1459+716')
#        
#        print(f"\nSEFD Results for J1459+716:")
#        print(f"Mean SEFD: {results_j1459['mean_sefd']:.1f} Jy")
#        print(f"Frequency: {results_j1459['frequency']/1e9:.3f} GHz")
#        print(f"Number of antennas: {len(results_j1459['antenna_sefd'])}")
#        
#        # Plot results
#        calc.plot_sefd_results(results_j1459, 'sefd_j1459.png')
#        
#        # Print per-antenna SEFD summary
#        print("\nPer-antenna SEFD summary:")
#        for ant in results_j1459['antenna_sefd'][:10]:  # Show first 10
#            if not np.isnan(ant['sefd']):
#                print(f"  Antenna {ant['antenna']}: {ant['sefd']:.1f} Jy")
#        
#    except Exception as e:
#        print(f"Error: {e}")
#        print("Make sure to update the MS file paths in the script!")
#        raise
#    
#    # You can also analyze the holography data
#    # calc.analyze_holography_data(ms_j0542, 'J0542+498')
#    # calc.analyze_holography_data(ms_j0319, 'J0319+415')
#    
#    # Compare results from different calibrators
#    # This helps identify systematic effects
#    
#    return results_j1459
#
#
#if __name__ == "__main__":
#    results = main()

In [4]:
#def main():
"""Main execution function"""
# Initialize calculator
calc = DSA110_SEFD_Calculator()

# Define your MS files
ms_j1459 = 'J1459_250529_60min_centered.ms'  # Replace with actual path
ms_j0542 = '2025-05-29_0319+415.ms'  # Replace with actual path
ms_j0319 = '2025-05-29_0542+498.ms'  # Replace with actual path

# Calculate SEFD from J1459+716 (1-hour track)
results_j1459 = calc.calculate_sefd_from_calibrator(ms_j1459, 'J1459+716')

print(f"\nSEFD Results for J1459+716:")
print(f"Mean SEFD: {results_j1459['mean_sefd']:.1f} Jy")
print(f"Frequency: {results_j1459['frequency']/1e9:.3f} GHz")
print(f"Number of antennas: {len(results_j1459['antenna_sefd'])}")

# Plot results
calc.plot_sefd_results(results_j1459, 'sefd_j1459.png')

# Print per-antenna SEFD summary
print("\nPer-antenna SEFD summary:")
for ant in results_j1459['antenna_sefd'][:10]:  # Show first 10
    if not np.isnan(ant['sefd']):
        print(f"  Antenna {ant['antenna']}: {ant['sefd']:.1f} Jy")


Calculating SEFD from J1459+716 observation

Getting observation info for J1459_250529_60min_centered.ms
Calibrator flux at 1.405 GHz: 1.83 Jy

SEFD Results for J1459+716:
Mean SEFD: 1.0 Jy
Frequency: 1.405 GHz
Number of antennas: 117
Saved plot to sefd_j1459.png

Per-antenna SEFD summary:
  Antenna 0: 1.0 Jy
  Antenna 1: 1.0 Jy
  Antenna 2: 1.0 Jy
  Antenna 3: 1.0 Jy
  Antenna 4: 1.0 Jy
  Antenna 5: 1.0 Jy
  Antenna 6: 1.0 Jy
  Antenna 7: 1.0 Jy
  Antenna 8: 1.0 Jy
  Antenna 9: 1.0 Jy


In [ ]:
gaincal( vis=msfile, caltable="gcal.0542", field=fluxfld,
         gaintype="G", solint="int", refant="0")

applycal(vis=msfile, gaintable=["gcal.0542"])

In [4]:
import numpy as np
from casatools import ms

def make_masks(msfile):
    m = ms(); m.open(msfile)
    data = m.getdata(['CORRECTED_DATA'])['corrected_data']
    amp  = np.abs(data).mean(axis=(0,1))   # avg over pol+chan
    on   = amp > 0.5 * amp.max()           # within ~⅓ FWHM
    off  = amp < 0.05 * amp.max()          # well off source
    m.close()
    return on, off


In [5]:
def sefd_per_baseline(msfile, off_mask):
    m = ms(); m.open(msfile)
    dat = m.getdata(['CORRECTED_DATA',
                     'CHAN_FREQ', 'INTERVAL'])           # full arrays
    vis_off = dat['corrected_data'][:,:,off_mask]
    sigma   = vis_off.view(np.float64).std()             # σ_ij
    dt      = dat['interval'][0]                         # s
    dnu     = np.diff(dat['chan_freq'][:,0]).mean()      # Hz
    sefd_bl = sigma * np.sqrt(2*dnu*dt)                  # Jy per baseline
    ant1, ant2 = m.getdata(['ANTENNA1','ANTENNA2']).values()
    m.close()
    return sefd_bl, ant1, ant2


In [6]:
import itertools, numpy.linalg as la

def invert_to_ant(sefd_bl, ant1, ant2):
    n_ant = max(ant1.max(), ant2.max()) + 1
    pairs = list(itertools.combinations(range(n_ant), 2))
    A = np.zeros((len(pairs), n_ant))
    for k,(i,j) in enumerate(pairs): A[k,i] = A[k,j] = 0.5
    sefd_ant = np.exp(la.lstsq(A, np.log(sefd_bl),
                               rcond=None)[0])            # Jy/antenna
    return sefd_ant


In [7]:
med = np.median(sefd_ant)
print(f"Median single‑dish SEFD ≈ {med:.0f} Jy (Expected ≈ 630 Jy)")


NameError: name 'sefd_ant' is not defined